In [ ]:
# !pip install -q pyspark
# !pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq
# !pip install -q graphframes
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
# spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
# !wget -N -P $spark_jars $graphframes_jar

In [ ]:
# from tqdm import tqdm
# import numpy as np
# from collections import Counter
# from threading import Thread

# import inverted_index_gcp
import inverted_index_gcp
import math
import pickle
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import time
from inverted_index_gcp import MultiFileReader, InvertedIndex
import struct
import time
import os
from google.cloud import storage
from contextlib import closing
import pickle as pkl

import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
# from time import time
# from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()


In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'myfirstgcp-370210'
!gcloud config set project {project_id}

data_bucket_name = 'elad_318640828'
try:
    if os.environ["elad_318640828"] is not None:
        pass  
except:
      !mkdir wikidumps
      # !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists


In [ ]:
# !gsutil cp -R gs://elad_318640828/small_postings .

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pickle

# Replace <path/to/dictionary.pkl> with the path to your dictionary file
with open('./small_postings/index_body.pkl', 'rb') as f:
    index_body = pickle.load(f)

print(index_body.posting_locs)

defaultdict(<class 'list'>, {'show': [('0_000.bin', 0)], 'election': [('0_000.bin', 2178)], '7th': [('0_000.bin', 2640)], 'places': [('0_000.bin', 3036)], 'thing': [('0_000.bin', 4194)], 'largely': [('0_000.bin', 4752)], 'key': [('0_000.bin', 5892)], 'arab': [('0_000.bin', 7182)], 'eggs': [('0_000.bin', 7812)], 'defines': [('0_000.bin', 8190)], 'pages': [('0_000.bin', 8508)], 'dynamic': [('0_000.bin', 8958)], 'smoke': [('0_000.bin', 9294)], 'organisation': [('0_000.bin', 9696)], 'biological': [('0_000.bin', 10038)], 'received': [('0_000.bin', 10638)], 'secondary': [('0_000.bin', 12666)], 'blend': [('0_000.bin', 13398)], 'explained': [('0_000.bin', 13728)], 'powers': [('0_000.bin', 14340)], 'leaving': [('0_000.bin', 14922)], 'index': [('0_000.bin', 16008)], 'actions': [('0_000.bin', 16704)], 'participants': [('0_000.bin', 17352)], 'record': [('0_000.bin', 17862)], 'colour': [('0_000.bin', 19164)], 'periods': [('0_000.bin', 19554)], 'emergency': [('0_000.bin', 20244)], 'prevent': [('0_00

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS

In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    print(inverted.posting_locs)
    locs = inverted.posting_locs[w]
    print(locs)
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
path_to_body_II = "./posting_gcp/"
query = "build"
def search_body(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
  # BEGIN SOLUTION
    with_stop_words_query_tokens = [token.group() for token in RE_WORD.finditer(query.lower())]

    query_tokens = []
    for tok in with_stop_words_query_tokens:
        if tok not in all_stopwords:
            query_tokens.append(tok)
    query_tokens = Counter(query_tokens)

    # this is normaliztion of the query for the cosine similarity
    norm_factor = 0
    for term, freq in query_tokens.items():
      norm_factor += freq**2
    norm_factor = math.sqrt(norm_factor)
    print(norm_factor)

    sim_dict = {}
    for term, freq in query_tokens.items():
      posting_list_per_term = read_posting_list(index_body, term)
      for doc_id, score in posting_list_per_term:
        norm_factor_doc = 0
        if doc_id in sim_dict:
          sim_dict[doc_id] += score
        else:
          sim_dict[doc_id] = score
        norm_factor_doc += score**2
      norm_factor_docs_final = math.sqrt(norm_factor_doc)

    similarity_final_dict = dict(sorted(sim_dict.items(), key= lambda x: (x[1]/(norm_factor * norm_factor_docs_final)), reverse= True))

    res = list()
    iteration = 0
    for doc_id in similarity_final_dict.keys():
        if iteration < 100:
            res.append((doc_id, index_body.id2titles[doc_id]))
            iteration += 1
        else:
            break
  # END SOLUTION
    return res



search_body(query)

1.0
defaultdict(<class 'list'>, {'show': [('0_000.bin', 0)], 'election': [('0_000.bin', 2178)], '7th': [('0_000.bin', 2640)], 'places': [('0_000.bin', 3036)], 'thing': [('0_000.bin', 4194)], 'largely': [('0_000.bin', 4752)], 'key': [('0_000.bin', 5892)], 'arab': [('0_000.bin', 7182)], 'eggs': [('0_000.bin', 7812)], 'defines': [('0_000.bin', 8190)], 'pages': [('0_000.bin', 8508)], 'dynamic': [('0_000.bin', 8958)], 'smoke': [('0_000.bin', 9294)], 'organisation': [('0_000.bin', 9696)], 'biological': [('0_000.bin', 10038)], 'received': [('0_000.bin', 10638)], 'secondary': [('0_000.bin', 12666)], 'blend': [('0_000.bin', 13398)], 'explained': [('0_000.bin', 13728)], 'powers': [('0_000.bin', 14340)], 'leaving': [('0_000.bin', 14922)], 'index': [('0_000.bin', 16008)], 'actions': [('0_000.bin', 16704)], 'participants': [('0_000.bin', 17352)], 'record': [('0_000.bin', 17862)], 'colour': [('0_000.bin', 19164)], 'periods': [('0_000.bin', 19554)], 'emergency': [('0_000.bin', 20244)], 'prevent': [('

[(36971117, 'Hyperloop'),
 (67055, 'Iron Man'),
 (909036, 'Elon Musk'),
 (2593693, 'History of Apple Inc.'),
 (51237650, 'History of Tesla, Inc.'),
 (53215263, 'The Boring Company'),
 (7771171, 'Java version history'),
 (856, 'Apple Inc.'),
 (832774, 'SpaceX'),
 (26976, 'Silicon Valley'),
 (27848, 'Steve Wozniak'),
 (45470, 'Manhattan'),
 (175537, 'Netflix'),
 (21179478, 'Social media marketing'),
 (45111627, 'Starlink'),
 (46734540, 'Construction of electronic cigarettes'),
 (1344, 'Apple I'),
 (1998, 'Austin, Texas'),
 (14580, 'Indian Ocean'),
 (42737, 'Dutch East India Company'),
 (970755, 'LinkedIn'),
 (1242998, 'History of Detroit'),
 (3356874, 'Paul Terrell'),
 (5533631, 'Tesla, Inc.'),
 (5676692, 'Iron Man (2008 film)'),
 (24470328, 'Learning through play'),
 (27306717, 'Marvel Cinematic Universe'),
 (41677925, 'Black Panther (film)'),
 (44254295, 'Avengers: Endgame'),
 (51430647, 'Music of the Marvel Cinematic Universe'),
 (2116, 'Apple II series'),
 (21652, 'Natural language p

In [ ]:
TUPLE_SIZE_BODY = 8

with open('', 'rb') as f:
    data = pickle.load(f)
# print(data.df)

FileNotFoundError: ignored